## Importing the Data

In [1]:
#TODO 分validation 
#方法1先分两部分, total training set + test set
#total training set 真正输入到cross validation function
#cross validation 中再划分training set和validation set 
#用找到的hyperparameter 训练
#方法2直接分成三部分
# Constants
#(TODO: cross validation, 得到最好模型, 用参数re training, 得到kappa score, 重复100次, 在test set上的kappa score
#2.增加additional features)
import warnings
warnings.filterwarnings("ignore") 

DATASET_DIR = './'
GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

import os
import pandas as pd

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
print(X.columns)
y = X["target_score"]
X = X.dropna(axis=1)
X = X.drop(['rater1_domain1', 'rater2_domain1'], axis=1)
essays = X['essay']




Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'essay_id', 'topic', 'essay', 'rater1_domain1',
       'rater2_domain1', 'rater3_domain1', 'target_score', 'rater1_domain2',
       'rater2_domain2', 'topic2_target', 'rater1_trait1', 'rater1_trait2',
       'rater1_trait3', 'rater1_trait4', 'rater1_trait5', 'rater1_trait6',
       'rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4',
       'rater2_trait5', 'rater2_trait6', 'rater3_trait1', 'rater3_trait2',
       'rater3_trait3', 'rater3_trait4', 'rater3_trait5', 'rater3_trait6',
       'err_ratio', 'avg_len', 'total_err_ratio', 'avg_wdlen', 'count_vector'],
      dtype='object')


In [2]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 14 columns):
Unnamed: 0            12976 non-null int64
Unnamed: 0.1          12976 non-null int64
Unnamed: 0.1.1        12976 non-null int64
Unnamed: 0.1.1.1      12976 non-null int64
Unnamed: 0.1.1.1.1    12976 non-null int64
essay_id              12976 non-null int64
topic                 12976 non-null int64
essay                 12976 non-null object
target_score          12976 non-null int64
err_ratio             12976 non-null object
avg_len               12976 non-null float64
total_err_ratio       12976 non-null float64
avg_wdlen             12976 non-null float64
count_vector          12976 non-null object
dtypes: float64(3), int64(8), object(3)
memory usage: 1.4+ MB


Minimum and Maximum Scores for each essay set.

In [3]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [4]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    #用np.append
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

## training Word2Vec model 

In [5]:
%%time
sentences = []

for essay in essays:
        # Obtaining all sentences from the training essays.
        sentences += essay_to_sentences(essay, remove_stopwords = True)

# Initializing variables for word2vec model.
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
# Generate word vectors.
word_list = []

for essay in essays:
    word_list.append(essay_to_wordlist(essay, remove_stopwords=True))
word_vec = getAvgFeatureVecs(word_list, model, num_features)
print(word_vec)


Training Word2Vec Model...
[[-0.02786942 -0.00171423 -0.03275253 ...  0.01196813  0.00433079
  -0.00623798]
 [-0.02451759 -0.00259538 -0.03267983 ...  0.01786377  0.0172
  -0.01459291]
 [-0.02719878 -0.00698547 -0.03992522 ...  0.0194615   0.00297113
   0.00032748]
 ...
 [-0.01948543 -0.0254215  -0.02861034 ... -0.02372727 -0.00529466
   0.00498873]
 [-0.00908957 -0.030986   -0.0323054  ... -0.02946295 -0.01291632
  -0.00324133]
 [-0.00824955 -0.00629606 -0.05475423 ... -0.00370063 -0.02395535
  -0.00758748]]
Wall time: 1min 25s


## split dataset

In [6]:
from sklearn.model_selection import train_test_split
X_train_validation, X_test, y_train_validation, y_test = train_test_split(word_vec, y, test_size=0.2, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.2, random_state=42)

In [7]:
print(np.array(word_vec).shape)
print(np.array(X_train).shape)
print(np.array(X_validation).shape)
print(np.array(X_test).shape)

(12976, 300)
(8304, 300)
(2076, 300)
(2596, 300)


## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [8]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from numpy.random import rand
# Import HyperOpt Library
from hyperopt import tpe, hp, fmin
def get_model(hidden_units_1, hidden_units_2, dropout_rate,num_features):
        """Define the model."""
        model = Sequential()
        model.add(LSTM(hidden_units_1, dropout=dropout_rate, recurrent_dropout=0.4, input_shape=[1, num_features], return_sequences=True))
        model.add(LSTM(hidden_units_2, recurrent_dropout=0.4))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='relu'))

        model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
        model.summary()

        return model
   


Using TensorFlow backend.


### tuning hyper parameters:

1.hidden units in 2 units, first:100-1000, s=10 second:20-500, s=10
2.dropout rates:randomly choose from 0-0.99
3.batch_size=16,32,64,128,256, 
4.epochs=1-10.step=1

In [9]:
def train_LSTM_model_for_tuning(params):
    hidden_units_1, hidden_units_2, dropout_rate, batch_size, epochs = params['hidden_units_1'],params['hidden_units_2'], params['dropout_rate'],params['batch_size'], params['epochs']
    

    results = []
    y_pred_list = []

    count = 1
    
    trainDataVecs = np.array(X_train)
    validationDataVecs = np.array(X_validation)
    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    validationDataVecs = np.reshape(validationDataVecs, (validationDataVecs.shape[0], 1, validationDataVecs.shape[1]))

    lstm_model = get_model(hidden_units_1, hidden_units_2, dropout_rate,num_features)#TODO更改num_features
    lstm_model.fit(trainDataVecs, y_train, batch_size=batch_size, epochs=epochs)
    #epochs 太大容易overfit
    #lstm_model.load_weights('./model_weights/final_lstm.h5')
    y_pred = lstm_model.predict(validationDataVecs)

    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)

    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_validation,y_pred,weights='quadratic')
    print("Kappa Score: {}".format(result))
    return result

ep = np.arange(1,11,1)

space = {
    #'num_features': hp.choice('num_features',np.arange(100,3001,100)),
    'hidden_units_1': hp.choice('hidden_units_1',np.arange(100,1001,10)),
    'hidden_units_2': hp.choice('hidden_units_2',np.arange(20,501,10)),
    'dropout_rate': hp.choice('dropout_rate',rand(10)),
    'batch_size': hp.choice('batch_size',[16,32,64,128,256]),
    'epochs': hp.choice('epochs',np.arange(1,11,1)),
}
best = fmin(
    fn=train_LSTM_model_for_tuning, # Objective Function to optimize
    space=space, # Hyperparameter's Search Space
    algo=tpe.suggest, # Optimization algorithm (representative TPE)
    max_evals=10 # Number of optimization attempts
)
print(best)

Model: "sequential_1"                                 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 290)            685560    
_________________________________________________________________
lstm_2 (LSTM)                (None, 330)               819720    
_________________________________________________________________
dropout_1 (Dropout)          (None, 330)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 331       
Total params: 1,505,611                               
Trainable params: 1,505,611                           
Non-trainable params: 0                               
_________________________________________________________________
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages

 - ETA: 1s - loss: 24.0610 - mae: 3.2018              
                                                     
1920/8304 [=====>........................]            
 - ETA: 1s - loss: 24.2550 - mae: 3.2159              
                                                     
2112/8304 [======>.......................]            
 - ETA: 1s - loss: 24.2527 - mae: 3.2250              
                                                     
2304/8304 [=======>......................]            
 - ETA: 1s - loss: 24.5798 - mae: 3.2447              
                                                     
2496/8304 [========>.....................]            
 - ETA: 1s - loss: 24.2915 - mae: 3.2215              
                                                     
2688/8304 [========>.....................]            
 - ETA: 1s - loss: 24.4329 - mae: 3.2267              
                                                     
2944/8304 [=========>....................]            
 - ETA: 1s - los

2560/8304 [========>.....................]                                     
 - ETA: 2s - loss: 112.0107 - mae: 6.0708                                      
                                                                               
2816/8304 [=========>....................]                                     
 - ETA: 2s - loss: 109.1706 - mae: 5.9806                                      
                                                                               
3328/8304 [===========>..................]                                     
 - ETA: 1s - loss: 104.9698 - mae: 5.8037                                      
                                                                               
3584/8304 [===========>..................]                                     
 - ETA: 1s - loss: 101.2629 - mae: 5.6754                                      
                                                                               
3840/8304 [============>................

 768/8304 [=>............................]                                     
 - ETA: 1s - loss: 34.8436 - mae: 3.6040                                       
                                                                              
1280/8304 [===>..........................]                                     
 - ETA: 1s - loss: 33.9354 - mae: 3.5891                                       
                                                                              
1792/8304 [=====>........................]                                     
 - ETA: 1s - loss: 34.0525 - mae: 3.5979                                       
                                                                              
2304/8304 [=======>......................]                                     
 - ETA: 0s - loss: 33.7801 - mae: 3.5901                                       
                                                                              
2816/8304 [=========>....................]  

4352/8304 [==============>...............]                                     
 - ETA: 0s - loss: 29.3329 - mae: 3.5468                                       
                                                                              
4608/8304 [===============>..............]                                     
 - ETA: 0s - loss: 29.7006 - mae: 3.5648                                       
                                                                              
4864/8304 [================>.............]                                     
 - ETA: 0s - loss: 29.7681 - mae: 3.5812                                       
                                                                              
5120/8304 [=================>............]                                     
 - ETA: 0s - loss: 29.9897 - mae: 3.5934                                       
                                                                              
5376/8304 [==================>...........]  

...

4096/8304 [=============>................]                                      
 - ETA: 2s - loss: 22.2428 - mae: 3.2086                                        
                                                                               
4224/8304 [==============>...............]                                      
 - ETA: 2s - loss: 22.3651 - mae: 3.2132                                        
                                                                               
4352/8304 [==============>...............]                                      
 - ETA: 2s - loss: 22.3608 - mae: 3.2111                                        
                                                                               
4480/8304 [===============>..............]                                      
 - ETA: 1s - loss: 22.3294 - mae: 3.2109                                        
                                                                               
4608/8304 [===============>.....

 - ETA: 2s - loss: 22.5948 - mae: 3.1970                                        
                                                                               
4480/8304 [===============>..............]                                      
 - ETA: 2s - loss: 22.8664 - mae: 3.2129                                        
                                                                               
4608/8304 [===============>..............]                                      
 - ETA: 2s - loss: 22.7849 - mae: 3.2117                                        
                                                                               
4736/8304 [================>.............]                                      
 - ETA: 1s - loss: 22.6383 - mae: 3.2024                                        
                                                                               
4864/8304 [================>.............]                                      
 - ETA: 1s - loss: 22.6702 - mae

                                                                               
4736/8304 [================>.............]                                      
 - ETA: 2s - loss: 22.1306 - mae: 3.1961                                        
                                                                               
4864/8304 [================>.............]                                      
 - ETA: 1s - loss: 22.1282 - mae: 3.2001                                        
                                                                               
4992/8304 [=================>............]                                      
 - ETA: 1s - loss: 22.1924 - mae: 3.2029                                        
                                                                               
5120/8304 [=================>............]                                      
 - ETA: 1s - loss: 21.9517 - mae: 3.1895                                        
                                

4992/8304 [=================>............]                                      
 - ETA: 1s - loss: 21.0448 - mae: 3.1194                                        
                                                                               
5120/8304 [=================>............]                                      
 - ETA: 1s - loss: 20.8126 - mae: 3.1049                                        
                                                                               
5248/8304 [=================>............]                                      
 - ETA: 1s - loss: 21.0053 - mae: 3.1141                                        
                                                                               
5376/8304 [==================>...........]                                      
 - ETA: 1s - loss: 20.9938 - mae: 3.1115                                        
                                                                               
5504/8304 [==================>..

 - ETA: 1s - loss: 20.3379 - mae: 3.0655                                        
                                                                               
5376/8304 [==================>...........]                                      
 - ETA: 1s - loss: 20.3395 - mae: 3.0642                                        
                                                                               
5504/8304 [==================>...........]                                      
 - ETA: 1s - loss: 20.2437 - mae: 3.0547                                        
                                                                               
5632/8304 [===================>..........]                                      
 - ETA: 1s - loss: 20.3328 - mae: 3.0631                                        
                                                                               
5760/8304 [===================>..........]                                      
 - ETA: 1s - loss: 20.3057 - mae

## train LSTM model using test set

In [10]:
hyperparameters = {'batch_size': 4, 'dropout_rate': 6, 'epochs': 8, 'hidden_units_1': 72, 'hidden_units_2': 48}
hidden_units_1, hidden_units_2, dropout_rate, batch_size, epochs = hyperparameters['hidden_units_1'],hyperparameters['hidden_units_2'], hyperparameters['dropout_rate'],hyperparameters['batch_size'], hyperparameters['epochs']


results = []
y_pred_list = []

count = 1

train_validation_DataVecs = np.array(X_train_validation)
testDataVecs = np.array(X_test)
# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
train_validation_DataVecs  = np.reshape(train_validation_DataVecs , (train_validation_DataVecs.shape[0], 1, train_validation_DataVecs.shape[1]))
testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))

lstm_model = get_model(hidden_units_1, hidden_units_2, dropout_rate,num_features)
lstm_model.fit(train_validation_DataVecs, y_train_validation, batch_size=batch_size, epochs=epochs)
#epochs 太大容易overfit
#lstm_model.load_weights('./model_weights/final_lstm.h5')
y_pred = lstm_model.predict(testDataVecs)

# Round y_pred to the nearest integer.
y_pred = np.around(y_pred)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 1, 72)             107424    
_________________________________________________________________
lstm_22 (LSTM)               (None, 48)                23232     
_________________________________________________________________
dropout_11 (Dropout)         (None, 48)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 49        
Total params: 130,705
Trainable params: 130,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/8
10380/10380 [==============================] - 10s 922us/step - loss: 44.3478 - mae: 3.5650
Epoch 2/8
10380/10380 [==============================] - 9s 888us/step - loss: 25.4536 - mae: 2.7733
Epoch 3/8
10380/10380 [===================

In [11]:
# Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
QWK = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("QWK: {}".format(QWK))
accuracy = accuracy_score(y_test, y_pred)
print("accuracy_score: {}".format(accuracy))

QWK: 0.9451906409923662
accuracy_score: 0.3208782742681048


In [12]:
from keras.models import load_model

lstm_model.save('AES_model_without_manually crafted features.h5')

In [13]:
# returns a compiled model
# identical to the previous one
lstm_model = load_model('AES_model_without_manually crafted features.h5')